# 证明带Relu的两层神经网络可以拟合任意的函数

## UAT定理

根据“万能逼近定理”（Universal Approximation Theorem）：一个前馈神经网络（feedforward neural network）如果具有线性输出层，同时至少存在一层具有任何一种“挤压”性质的激活函数（例如logistic sigmoid激活函数）的隐藏层，那么只要给予这个网络足够数量的隐藏单元，它就可以以任意的精度来近似任何从一个有限维空间到另一个有限维空间的波莱尔可测函数(Borel Measurable Function)。

由于ReLU是一种非线性激活函数，可以引入网络的非线性特性，使其具备强大的表示能力，从而正好满足万能逼近定理

## 形式上

UAT 指出，只要使用的激活函数是有界的、连续的和单调递增的，紧凑域上的任何连续函数都可以用只有一个隐藏层的神经网络来近似。

1. 连续函数f(x)，
2. 输入x的有限范围是[a,b]，并且
3. 期望近似精度ε>0，

那么存在一个神经网络，在[a,b]中的任何位置，能够近似f(x)，并且近似误差小于ε。

# 线性回归的从零开始实现


In [4]:
%matplotlib inline
import random
import torch

## 生成数据集

为了简单起见，我将[**根据带有噪声的线性模型构造一个人造数据集。**]
我的任务是使用这个有限样本的数据集来恢复这个模型的参数

在下面的代码中，我生成一个包含1000个样本的数据集，
每个样本包含从标准正态分布中采样的2个特征。
我的合成数据集是一个矩阵$\mathbf{X}\in \mathbb{R}^{1000 \times 2}$。

我使用线性模型参数$\mathbf{w} = [2, -3.4]^\top$、$b = 4.2$
和噪声项$\epsilon$生成数据集及其标签：

$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.$$


$\epsilon$可以视为模型预测和标签时的潜在观测误差。
在这里我认为标准假设成立，即$\epsilon$服从均值为0的正态分布。
为了简化问题，我将标准差设为0.01。
下面的代码生成合成数据集。


In [5]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [6]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

注意，[**`features`中的每一行都包含一个二维数据样本，
`labels`中的每一行都包含一维标签值（一个标量）**]。


In [7]:
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([-0.4241, -1.8610]) 
label: tensor([9.6864])


通过生成第二个特征`features[:, 1]`和`labels`的散点图，
可以直观观察到两者之间的线性关系。


## 读取数据集

回想一下，训练模型时要对数据集进行遍历，每次抽取一小批量样本，并使用它们来更新我的模型。
由于这个过程是训练机器学习算法的基础，所以有必要定义一个函数，
该函数能打乱数据集中的样本并以小批量方式获取数据。

在下面的代码中，我[**定义一个`data_iter`函数，
该函数接收批量大小、特征矩阵和标签向量作为输入，生成大小为`batch_size`的小批量**]。
每个小批量包含一组特征和标签。


In [8]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

## 初始化模型参数

[**在我开始用小批量随机梯度下降优化我的模型参数之前**]，
(**我需要先有一些参数**)。
在下面的代码中，我通过从均值为0、标准差为0.01的正态分布中采样随机数来初始化权重，
并将偏置初始化为0。


In [9]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [10]:
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b

## [**定义损失函数**]

在实现中，我需要将真实值`y`的形状转换为和预测值`y_hat`的形状相同。


In [11]:
def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

## (**定义优化算法**)
在每一步中，使用从数据集中随机抽取的一个小批量，然后根据参数计算损失的梯度。
接下来，朝着减少损失的方向更新我的参数。
下面的函数实现小批量随机梯度下降更新。
该函数接受模型参数集合、学习速率和批量大小作为输入。每
一步更新的大小由学习速率`lr`决定。
因为我计算的损失是一个批量样本的总和，所以我用批量大小（`batch_size`）
来规范化步长，这样步长大小就不会取决于我对批量大小的选择。


In [12]:
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

## 训练

现在我已经准备好了模型训练所有需要的要素，可以实现主要的[**训练过程**]部分了。
理解这段代码至关重要，因为从事深度学习后，
相同的训练过程几乎一遍又一遍地出现。
在每次迭代中，我读取一小批量训练样本，并通过我的模型来获得一组预测。
计算完损失后，我开始反向传播，存储每个参数的梯度。
最后，我调用优化算法`sgd`来更新模型参数。

概括一下，我将执行以下循环：

* 初始化参数
* 重复以下训练，直到完成
    * 计算梯度$\mathbf{g} \leftarrow \partial_{(\mathbf{w},b)} \frac{1}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} l(\mathbf{x}^{(i)}, y^{(i)}, \mathbf{w}, b)$
    * 更新参数$(\mathbf{w}, b) \leftarrow (\mathbf{w}, b) - \eta \mathbf{g}$

在每个*迭代周期*（epoch）中，我使用`data_iter`函数遍历整个数据集，
并将训练数据集中所有样本都使用一次（假设样本数能够被批量大小整除）。
这里的迭代周期个数`num_epochs`和学习率`lr`都是超参数，分别设为3和0.03。
设置超参数很棘手，需要通过反复试验进行调整。
我现在忽略这些细节，以后会在 :numref:`chap_optimization`中详细介绍。


In [15]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
batch_size = 10

In [16]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.042858
epoch 2, loss 0.000163
epoch 3, loss 0.000048


因为我使用的是自己合成的数据集，所以我知道真正的参数是什么。
因此，我可以通过[**比较真实参数和通过训练学到的参数来评估训练的成功程度**]。
事实上，真实参数和通过训练学到的参数确实非常接近。


In [17]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 0.0012, -0.0006], grad_fn=<SubBackward0>)
b的估计误差: tensor([-0.0001], grad_fn=<RsubBackward1>)


[Discussions](https://discuss.d2l.ai/t/1778)
